# Classify content in markdown format in three categories, to detect low-value content

This code demonstrate how to classify markdown content in three categories:
+ TABLE-ONLY: If there is only a table with no context nor description.
+ TEXT-ONLY: if there is no table at all.
+ TABLE-WITH-CONTEXT: if there is a table with context or description

The output is one those three categories and the explaination of the reason to classify the content on it.

## Prerequisites

+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
+ An Azure OpenAI service with the service name and an API key.
+ A deployment of GPT-4o in the on the Azure OpenAI Service.

We used Python 3.12.5, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install openai

## Import packages and create AOAI client

In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

import sys
sys.path.append('../..')
from rag_utils import load_files, call_aoai

# Load environment variables from .env
load_dotenv(override=True)

# AOAI FOR CLASSIFICATION
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_apikey = os.environ["AZURE_OPENAI_API_KEY"]
aoai_model_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
# Create AOAI client
aoai_api_version = '2024-02-15-preview'
aoai_client = AzureOpenAI(
    azure_deployment=aoai_model_name,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint,
    api_key=aoai_apikey
)

## Classify sections in three categories:
### TABLE-ONLY, TABLE-WITH-CONTEXT and TEXT-ONLY

In [2]:
def classify_with_gpt4o(text):

    system_prompt = "You have to detect in this document a table and identify if there is any context or description that describes the meaning of the table. The context could be inside or outside of the table. The context will be a sentence with several or paragraph. If there is only a table with no context nor description return 'TABLE-ONLY', if there is no table at all return 'TEXT-ONLY', and if there is a table with context or description return 'TABLE-WITH-CONTEXT'. Add the explaination of your decision. Your answer must be with this format, one line per document: Type, Explaination."
    
    user_prompt = f'Document: "{text}"'
    
    return call_aoai(aoai_client, aoai_model_name, system_prompt, user_prompt, 0.5, 4096)

In [ ]:
# Test 1: TABLE-ONLY

markdown = """
"Codificaci\u00f3n ATOS\n===\n\n||\n| - |\n| Gran P\u00fablico - Servicios / Transmisi\u00f3n de datos MS-Activa / GPRS - Servicios / E-moci\u00f3n MS-Activa / Accesos / GPRS Empresas - Servicios/ Transmisi\u00f3n de datos / GPRS |\n| |\n"
"""

result = classify_with_gpt4o(markdown)
print(result)

In [ ]:
# Test 2: TABLE-WITH-CONTEXT

markdown = """
"Plantillas\n===\n\n||\n| - |\n| Patrocinio de Eventos Ante llamadas de clientes que indiquen estar interesados en que Telef\u00f3nica M\u00f3viles patrocine un evento que su empresa esta preparando, indicar que deben enviar por correo un dossier completo con todos los datos a: Att Francisco Garc\u00eda del Pozo o Roc\u00edo Vallejo-Nagera Telef\u00f3nica M\u00f3viles Espa\u00f1a Distrito C, Edificio Sur 1, Planta 4 C/ Ronda de la Comunicaci\u00f3n s/n 28050 Madrid |\n| |\n"
"""

result = classify_with_gpt4o(markdown)
print(result)

In [ ]:
# Test 3: TEXT-ONLY

markdown = """
"Esta llamada se codifica:\n===\n\nRuta :\n\nGenerar Gesti\u00f3n - Tramitaci\u00f3n - Servicios - L\u00ednea M\u00faltiple - Multisim - Alta/Baja\n\nSe debe seleccionar la \u00ednea que se va a modificar y se habilitar\u00e1 el bot\u00f3n \"Modificar\" Al acceder a la pantalla de modificaci\u00f3n se debe seleccionar como \"Tipo de Actuaci\u00f3n\": BAJA y \"Aceptar\"\n"
"""

result = classify_with_gpt4o(markdown)
print(result)

## Classify every txt file in the input directory

In [ ]:
# Chunk markdown files and write the chunks as files in the output directory
input_dir = '../../data_out/markdown_files'
markdown_contents = load_files(input_dir, '.txt')

table_only=0
table_with_content=0
text_only=0
for i, markdown_content in enumerate(markdown_contents):
    print(f"[{i + 1}]: title: {markdown_content['title']}")

    result = classify_with_gpt4o(markdown_content['content'])
    print(f'\t {result}')

    if 'TABLE-ONLY' in result:
        table_only+=1
    elif 'TABLE-WITH-CONTEXT' in result:
        table_with_content+=1
    elif 'TEXT-ONLY' in result:
        text_only+=1

print(f'Total number of "TABLE-ONLY": {table_only}')
print(f'Total number of "TABLE-WITH-CONTEXT": {table_only}')
print(f'Total number of "TEXT-ONLY": {text_only}')
